# Function Setup

### 1.) Classical Life Insurance Calculation

In [2]:
# Calculate annuity of term life insurance
def get_termlife_annuity(age, duration, interest, A= 0.00022, B=2.7*10**(-6), c=1.124):
    
    v = 1/(1+interest)
    ann = 0
    for k in range(duration):
        ann+= v**k*np.exp(-A*k-B/np.log(c)*c**(age)*(c**k-1))                  
    
    return ann

In [3]:
# Calculate apv of benefits for term life insurance
def get_termlife_APV_ben(age, duration, interest, A= 0.00022, B=2.7*10**(-6), c=1.124):
    
    v = 1/(1+interest)
    apv = 0
    for k in range(duration):
        apv += v**(k+1)*np.exp(-A*k-B/np.log(c)*c**(age)*(c**k-1))*(1-np.exp(-A-B/np.log(c)*c**(age+k)*(c-1)))
    
    return apv

In [4]:
# Calculate premium for term life insurance
def get_termlife_premium(age_init,Sum_ins,duration,  interest,  A= 0.00022, B=2.7*10**(-6), c=1.124):
    
    return Sum_ins*get_termlife_APV_ben(age_init, duration, interest, A, B, c)/get_termlife_annuity(age_init, duration, interest, A, B, c)

In [5]:
# Calculate current Policy value of term life insurance contract
def get_termlife_reserve(age_curr, Sum_ins, duration,  interest,age_of_contract=0, A= 0.00022, B=2.7*10**(-6), c=1.124):
    age_init = age_curr-age_of_contract
    prem = get_termlife_premium(age_init,Sum_ins, duration, interest, A,B,c)
    apv_prem = get_termlife_annuity(age_curr, duration-age_of_contract, interest, A,B,c)
    apv_ben = get_termlife_APV_ben(age_curr, duration-age_of_contract, interest, A,B,c)
    
    return Sum_ins*apv_ben -prem*apv_prem 

In [2]:
# Calculate Policy values up to maturity of term life insurance contract
# Potentially even including the past policy values, starting at the start of the contract
def get_termlife_reserve_profile(age_curr, Sum_ins, duration, interest,age_of_contract = 0,  A= 0.00022, 
                                 B=2.7*10**(-6), c=1.124, option_past = True, age_limit = 120):
    
    age_init = age_curr-age_of_contract
    premium = get_termlife_premium(age_init, Sum_ins,duration, interest, A,B,c)
    reserve = np.zeros(duration+1)
    
    # No expenses so far
    e_ann = 0
    e_init = 0
    # No claims-related expenses
    E_claim = 0
    reserve[0] = - e_init
    reserve[-1] = 0
    for k in range(duration-1): # Exclude value of reserve at maturity of contract (set 0, to avoid inaccuracy due to rounding errors)
        if age_init+k < age_limit:
            prob_live = np.exp(-A-B/np.log(c)*c**(age_init+k)*(c-1))# at age x+k
            reserve[k+1] =((reserve[k]+premium-e_ann)*(1+interest) - (Sum_ins+E_claim)*(1-prob_live))/prob_live
        else:
            #reserve[k+1] = Sum_ins/(1+interest)+(e_ann+e_init)/(1+interest)-premium*1
            reserve[k+1] = (Sum_ins+(e_ann+e_init))*get_termlife_APV_ben(age_init+k,duration,interest)-premium*get_termlife_annuity(age_init+k, duration, interest)
        pass
    
    
    if option_past==False:
        if age_of_contract <= duration:
            reserve = reserve[age_of_contract:]
        else:
            reserve = 0

    return reserve   